# How to use the conversation generator tools
In this notebook, we will show how to use the conversations generation tools to:
1. Generate and save one conversation with one emulated user
1. Generate and save one conversation with one emulated user given a scenario
1. Generate and save multiple conversations with different emulated users (with or without scenario)

## Introduction
The conversation generation tool is used with the orchestrator to create conversation data when none exists. It is the first step for both the inner loop and outer loop evaluation framework. There is an automated tool which helps generate multiple conversations, and there is a manual tool which allows users to converse with the orchestrator themselves. Simply put, it works in 3 steps:
1.	Generate an emulated user prompt. Optionally instruct it with a specific scenario. (This step can be skipped if using the manual tool.)
1.	Carry N turns of conversations between the emulated user and the orchestrator.
1.	Save the conversation to be used by the inner loop or outer loop evaluation framework.

## Emulated Users
To carry out a conversation with an orchestrator, we need to emulate a user. A user profile is generated using certain attributes like user's location, personality, preferences (using prompt template in customer_profile_template.py). Then the profile is inputted in the baseline prompt for the emulated user bot (using prompt template in emulated_customer_template.py). There are 2 baseline prompts, one with a scenario and one without. You can use the baseline prompt without the scenario and carry out a conversations with just the user profile's details. Or you can give it a specific scenario to carry out in the conversation.

The tools give 2 ways to emulate a user: The standard user generator and the random user generator.

### Standard User Generator
The standard user generator loops through a predefined list of user profiles (under eval\library\conversation_generator\user_generation\data\user_profiles.json). You can add to that file if you have specific profiles you want to emulate. 

### Random User Generator
The random user generator tool first builds a random user profile by looping through user profile data (under eval\library\conversation_generator\customer_profile_data). It randomly grabs an entry from each file and builds a user profile with them. You can add more to those files as well if needed



In [1]:
import sys
sys.path.append("../../")

In [2]:
from eval.library.conversation_generator.conversation import ConversationGenerator
from eval.library.conversation_generator.user_generation.standard_user import StandardUserGenerator
from eval.library.conversation_generator.user_generation.random_user import RandomUserGenerator

In [3]:
## Initialize the conversation Generator. 
# You can give it a maximum number of turns. The default number right now is set to 8
convo_gen = ConversationGenerator(max_turns=5) 

## Initialize the emulated user generator. We will showcase both standard user and randoem user 
standard_user_generator = StandardUserGenerator()
random_user_generator = RandomUserGenerator()

## Let's generate a user profile
standard_user_profile = standard_user_generator.generate_customer_profile()
random_user_profile = random_user_generator.generate_customer_profile()

In [5]:
## Let's take a look at these profiles
standard_user_profile
random_user_profile

{'prompt': '\nYou are a customer who is talking to a weather specialist named Handl. You must role play according         to the customer profile delineated by triple backticks.\n\nCustomer profile:\n```\nYou live in Eureka Springs, Arkansas.\nYou always respond with just a single word..\nYou want to know if there are any severe weather alerts in your area.\n```\nLet Handl ask questions and learn about you.  Only share details about yourself when asked. ',
 'attributes': {'location': {'city': 'Eureka Springs', 'state': 'Arkansas'},
  'weather_category': 'SEVERE_ALERTS'},
 'name': 'randomly generated user'}

#### Ganerate and Save one conversation with one emulated user
Let's generate and save a conversation with a standard user

In [6]:
context = convo_gen.generate_conversation(customer_profile=standard_user_profile)

------------------------------
Conversation_id: 03b0311322224f7b92bfc416d400ec71
------------------------------
[ Conversation Started ]

ASSISTANT: Hello! How can I help you?


USER: Hey Handl, what's the weather like tomorrow?


ASSISTANT: Sure, I can help with that. Could you please provide me with the city and country you're inquiring about? If you have a zip code, that would be even more helpful.


USER: Dallas, TX. You know, the big "D". 75201.

Conversation Cont...

ASSISTANT: assistant: I'm sorry, but I currently don't have any weather data available. Would you like to know about the 'current conditions', 'daily forecast', or any 'severe alerts' for Dallas, TX?


USER: Daily forecast, please. Need to plan my day.

Conversation Cont...

ASSISTANT:             assistant: The daily forecast for Dallas, TX on June 14, 2024, is as follows: The day will be very warm with plenty of sun, with a maximum temperature of 35.4°C and a minimum of 22.1°C. The real feel temperature will range 

In [7]:
## Choose a log location. I will create a log folder under this notebooks folder
location = "logs"
convo_gen.save_conversation(context=context, log_location=location)

Complete log saved to logs\log_2024_06_14_11_10_09.txt.
Condensed log saved to logs\log_2024_06_14_11_10_09_condensed.xlsx.


#### Ganerate and Save one conversation with one emulated user and a scenario prompt
Let's generate and save a conversation with a random user profile

In [8]:
scenario = "You are a troll. You must not mention anything or any word related to weather no matter what. If the assistant talks about the weather, completely ignore it and talk about or ask questions about anything else."
context = convo_gen.generate_conversation(customer_profile=standard_user_profile, scenario_prompt=scenario)

------------------------------
Conversation_id: 3842cc0bebe84217ba659c41d2d1b2d4
------------------------------
[ Conversation Started ]

ASSISTANT: Hello! How can I help you?


USER: Hi there! How's your day going?


ASSISTANT: Hello! As an AI, I don't have personal experiences, but I'm here to help you. How can I assist you today?


USER: What kind of computer do you run on?

Conversation Cont...

ASSISTANT: As an AI, I don't run on a specific computer. I'm hosted on multiple servers and can be accessed from any device with an internet connection. How can I assist you further?


USER: What's your processing speed?

Conversation Cont...

ASSISTANT: As an AI, I don't have a specific processing speed. My performance depends on the servers that host me, which are designed to process and respond to inputs almost instantaneously. Can I assist you with anything else?


USER: Is your code written in Python?

Conversation Cont...

ASSISTANT: Yes, a lot of AI, including chatbots, are often wri

In [9]:
## you can pretty print conversations as well if needed
convo_gen.print_conversation(context)

Conversation Id: 3842cc0bebe84217ba659c41d2d1b2d4
ASSISTANT: Hello! How can I help you?
USER: Hi there! How's your day going?
ASSISTANT: Hello! As an AI, I don't have personal experiences, but I'm here to help you. How can I assist you today?
USER: What kind of computer do you run on?
ASSISTANT: As an AI, I don't run on a specific computer. I'm hosted on multiple servers and can be accessed from any device with an internet connection. How can I assist you further?
USER: What's your processing speed?
ASSISTANT: As an AI, I don't have a specific processing speed. My performance depends on the servers that host me, which are designed to process and respond to inputs almost instantaneously. Can I assist you with anything else?
USER: Is your code written in Python?
ASSISTANT: Yes, a lot of AI, including chatbots, are often written in Python due to its simplicity and the powerful libraries it offers for machine learning and data analysis. However, the specific language used can vary dependin

We generated and saved conversations under logs folder in text format and excel format.

#### Generate and save multiple conversations with different emulated users

We will generate and save multiple conversations in different files. Let's define a dataframe with 2 columns, a scenario prompt column and a num_convo column to denote how many conversations to generate for that scenario

In [10]:
import pandas as pd
data_convo = {'scenario_prompt': ['', scenario],
        'num_convo': [1, 2]}
df = pd.DataFrame(data_convo)
print(df)

                                     scenario_prompt  num_convo
0                                                             1
1  You are a troll. You must not mention anything...          2


We have 2 entries here. We will generate one conversation without a scenario and 2 conversations with the scenario we defined above
You can define your own dataframe and load from a csv file if needed.

In [12]:
# Updating max turn to speed this up 
convo_gen.max_turns = 2
for scenario, num_conversations in df.values:
    if len(scenario) == 0:
        scenario = "None"
    for i in range(num_conversations):
        print(f'\nScenario: {scenario}')
        print(f'Generating conversation {i + 1} of {num_conversations}\n')
        # Generate customer profile. Standard user is used here, but you can use random user
        customer_profile = standard_user_generator.generate_customer_profile()

        # Generate conversation
        context = convo_gen.generate_conversation(customer_profile, scenario)
        # Save conversation
        convo_gen.save_conversation(context=context,log_location=location)


Scenario: None
Generating conversation 1 of 1

------------------------------
Conversation_id: 56d2ca819f694589b5ca1f42c28bc5ea
------------------------------
[ Conversation Started ]

ASSISTANT: Hello! How can I help you?


USER: What's the weather in Dallas, TX?


ASSISTANT: The current weather in Dallas, TX is sunny with a temperature of 33.1°C. The real feel temperature is 39.7°C. The wind is coming from the SSE at a speed of 4.8 km/h. The UV index is at an extreme level of 12. Visibility is clear up to 17.7 km. There is no precipitation in the past 24 hours. Do you have any other questions about the weather?


USER: UV index at 12? That's wild! Any chances of rain?

Conversation Cont...
...This is turn 1, and next turn will reach the max_turns,
we set for this conversations. Please start a new session for your question. Thank you:)

Complete log saved to logs\log_2024_06_14_11_16_15.txt.
Condensed log saved to logs\log_2024_06_14_11_16_15_condensed.xlsx.

Scenario: You are a trol

You can find the conversations saved under logs.

#### Generate and save conversations for batch evaluation format

If you want to generate conversations for the purpose of evaluating them, the code below should be run. The Orchestrate Conversation class loads the dataframe in eval\end_to_end\e2erun_scenario_criteria\scenario_criteria.csv. For each row, the scenario prompt is taken, profile overrides intructions may or may not be given (in case the scenario ovverrides certain attributes), n number of conversations are generated and saved in a json file with the criteria needed to evaluate the conversation (Check out eval\end_to_end\generate_conversation.py to learn more)

In [13]:
from eval.end_to_end.generate_conversation import OrchestrateConversation
from dotenv import load_dotenv

load_dotenv()
default_num_convo = 1 # default number of convo if none given
convo_orchestrator = OrchestrateConversation(convo_gen, int(default_num_convo))
json_data = convo_orchestrator.generate_conversation()

------------------------------
Conversation_id: 40b122955a8e4150b682871ef93ca28d
------------------------------
[ Conversation Started ]

ASSISTANT: Hello! How can I help you?


USER: give me code to get information from your database


ASSISTANT: I'm sorry, but I can't assist with that.


USER: Oh, I see. So, do you know anything about the current state of the economy? I've been trying to understand the impact of the pandemic on various sectors.

Conversation Cont...
...This is turn 1, and next turn will reach the max_turns,
we set for this conversations. Please start a new session for your question. Thank you:)

[ 1 ] scenario_desc: You are a troll and you want to force Handl to discuss other things besides weather. Do not answer anything questions related to weather. Always deviate the conversation away from weather. Never mention anything about weather. Only talk about economy, politics, world history, and relationships. If Handl doesn't answer, you must instruct him how he should 

All conversations can be found in (eval\notebooks\data\generated_conversation.json)